In [ ]:
# evaluation.ipynb

# Import necessary libraries
import os
import pandas as pd
import json

# Import functions from utils.py
from utils import (
    calculate_meter_score,
    calculate_rhyme_score,
    compare_with_baselines,
    load_diffusion_model,
    load_transformer_model,
    generate_classical_poem
)

# Configure visualization settings if needed
%matplotlib inline


In [ ]:
# Define model paths and configurations
diffusion_model_path = '../models/diffusion/diffusion_model_final.h5'
transformer_model_path = '../models/transformers/transformer_model_final.h5'
transformer_model_name = 'aubmindlab/bert-base-arabertv2'  # Ensure this matches your trained model
max_length = 1000

# Load models
print("Loading trained models...")
transformer_model, tokenizer = load_transformer_model(transformer_model_path, transformer_model_name, max_length)
diffusion_model = load_diffusion_model(diffusion_model_path)
print("Models loaded successfully.")


In [ ]:
# Define input directory containing generated poems
generated_poems_dir = '../results/outputs'

# List all generated poem files
poem_files = [f for f in os.listdir(generated_poems_dir) if f.endswith('.txt')]
print(f"Found {len(poem_files)} generated poems.")

# Initialize results list
evaluation_results = []


In [ ]:
# Iterate through each generated poem and compute metrics
for filename in poem_files:
    file_path = os.path.join(generated_poems_dir, filename)
    with open(file_path, 'r', encoding='utf-8') as f:
        poem = f.read()
    
    # Compute meter and rhyme scores
    meter_score = calculate_meter_score(poem)
    rhyme_score = calculate_rhyme_score(poem)
    
    # Append to results
    evaluation_results.append({
        "filename": filename,
        "meter_score": meter_score,
        "rhyme_score": rhyme_score
    })

print("Computed evaluation metrics for all generated poems.")


In [ ]:
# Compare with baseline models
print("Comparing with baseline models...")
compare_with_baselines(evaluation_results)
print("Comparison with baselines completed.")


In [ ]:
# Define metrics output path
metrics_output_dir = '../data/metrics'
os.makedirs(metrics_output_dir, exist_ok=True)
evaluation_results_path = os.path.join(metrics_output_dir, 'evaluation_results.json')

# Save results as JSON
with open(evaluation_results_path, 'w', encoding='utf-8') as f:
    json.dump(evaluation_results, f, ensure_ascii=False, indent=2)

print(f"Evaluation results saved to {evaluation_results_path}")


In [ ]:
# Load evaluation results
with open(evaluation_results_path, 'r', encoding='utf-8') as f:
    results = json.load(f)

# Convert to DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display summary statistics
results_df.describe()


In [ ]:
# Plot distribution of meter scores
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.histplot(results_df['meter_score'], bins=20, kde=True)
plt.title('Distribution of Meter Scores')
plt.xlabel('Meter Score')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Plot distribution of rhyme scores
plt.figure(figsize=(12, 6))
sns.histplot(results_df['rhyme_score'], bins=20, kde=True)
plt.title('Distribution of Rhyme Scores')
plt.xlabel('Rhyme Score')
plt.ylabel('Frequency')
plt.show()
